In [37]:
"""
Authors      : Aditya Jain and Safwan Jamal
Date started : November 16, 2022
About        : Convex Optimization project; training script
"""

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn

from cifardataset import CIFARDataset
from custom_cnn_one import CustomCNN
from custom_cnn_two import CustomCNNTwo

#### Load Model

In [38]:
num_classes = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Available device is {device}')
model = CustomCNNTwo(num_classes).to(device)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)

Available device is cuda


#### Load Data

In [39]:
train_set      = './image_list_cifar/train_full.csv'
num_epochs     = 70
early_stopping = 8

In [40]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size     = 4
class_list     = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
                  'dog', 'frog', 'horse', 'ship', 'truck']
train_root_dir = './cifar-10-images/train/'
test_root_dir  = './cifar-10-images/test/'
test_set       = './image_list_cifar/test.csv'

train_data       = CIFARDataset(train_root_dir, train_set, class_list, transform)
train_dataloader = DataLoader(train_data,batch_size=batch_size, shuffle=True, num_workers=2)
print(len(train_dataloader))

test_data        = CIFARDataset(test_root_dir, test_set, class_list, transform)
test_dataloader  = DataLoader(test_data,batch_size=batch_size, shuffle=True, num_workers=2)

375


#### Load Optimizer and Loss function

In [41]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
loss_func = nn.CrossEntropyLoss()

#### Training

In [42]:
best_test_accuracy = 0.0
early_stop_count   = 0

for epoch in range(num_epochs): 
    # Model Training
    model.train()
    train_epoch_loss = 0.0
    for image_batch, label_batch in train_dataloader:    
        image_batch, label_batch = image_batch.to(device), label_batch.to(device)
        label_batch = label_batch.squeeze_()
        
        # Compute and apply gradients
        optimizer.zero_grad()
        outputs   = model(image_batch)  
        t_loss    = loss_func(outputs, label_batch)
        t_loss.backward()
        optimizer.step()
        train_epoch_loss += t_loss.item()        
        
    print(f'Training loss for epoch {epoch+1} is {train_epoch_loss/len(train_dataloader)}')
    
    # Model Evaluation
    model.eval()
    total_samples   = 0.0
    total_correct   = 0.0
    for image_batch, label_batch in test_dataloader:    
        image_batch, label_batch = image_batch.to(device), label_batch.to(device)  
        label_batch = label_batch.squeeze_()
        outputs = model(image_batch)
        
        # Calculate batch accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_samples += label_batch.size(0)
        total_correct += (predicted == label_batch).sum().item()
    curr_accuracy = (total_correct/total_samples)*100
    print(f'Test accuracy for epoch {epoch+1} is {curr_accuracy}%')
    
    if curr_accuracy > best_test_accuracy:
        best_test_accuracy = curr_accuracy
        print(f'Best test accuracy improved to {best_test_accuracy}%')
        early_stop_count = 0
    else:
        early_stop_count += 1
        
    if early_stop_count==early_stopping:
        print(f'The best test accuracy achieved is {best_test_accuracy}')
        break

Best test accuracy improved to 10.0%
Best test accuracy improved to 13.5%
Best test accuracy improved to 17.5%
Best test accuracy improved to 20.0%
Best test accuracy improved to 25.25%
Best test accuracy improved to 31.25%
Best test accuracy improved to 36.25%
Best test accuracy improved to 38.0%
Best test accuracy improved to 39.0%
Best test accuracy improved to 40.25%
Best test accuracy improved to 41.0%
The best test accuracy achieved is 41.0
